In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('/home/saloni/Documents/ML/Mortality/train.csv')
labels=pd.read_csv('/home/saloni/Documents/ML/Mortality/labels_train.csv')


In [4]:
dailydata=df.copy()
result=pd.concat([dailydata,labels],axis=1)

In [5]:
result.head(1)

,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,DiasABP,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,In-hospital_death
0,77.0,31.0,46.0,54,2.973333,10.5,0.7,154.0,0.75,58.795833,...,97.25,116.891892,37.357143,2.1,0.14,171.052632,10.3,80.060976,7.387273,0


In [6]:
list_of_variables = ['Age', 'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 'Creatinine',
                             'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'Height', 'HR', 'ICUType', 'K',
                             'Lactate', 'Mg', 'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 'PaO2',
                             'pH', 'Platelets', 'RecordID', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI','TroponinT','Urine',
                             'WBC', 'Weight']

In [7]:
datawithoutoutliers=result.copy()
    

In [8]:
for i in list_of_variables:
    datawithoutoutliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())<=(4*datawithoutoutliers[i].std())]
    outliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())>=(4*datawithoutoutliers[i].std())]
    
print(datawithoutoutliers.shape)

(3169, 43)


In [9]:
labels=datawithoutoutliers.pop("In-hospital_death")
df=datawithoutoutliers.copy()

In [10]:
labels.shape


(3169,)

In [11]:
df.shape

(3169, 42)

In [12]:
from sklearn.preprocessing import MinMaxScaler
#normalization
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)


In [13]:
from imblearn.over_sampling import SMOTE

df_res,labels_res=SMOTE(random_state=9).fit_sample(df,labels) #balancing the data
df_res=pd.DataFrame(df_res)
labels_res=pd.DataFrame(labels_res)

In [14]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
onehot_encoded = enc.fit_transform(labels_res)

labels_res=onehot_encoded

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_res,labels_res,test_size=0.33,random_state=42)


In [16]:
sess=tf.InteractiveSession()

In [17]:
n_hidden_1=120
n_hidden_2=120
n_hidden_3=120
n_hidden_4=120
n_hidden_5=120


x=tf.placeholder(tf.float32,shape=[None,42])
y_=tf.placeholder(tf.float32,shape=[None,2])
w =tf.Variable(tf.zeros([42,2]))
b=tf.Variable(tf.zeros([2]))

def multilayer_perceptron(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.sigmoid(layer_3)
    
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.sigmoid(layer_4)
    
    layer_5=tf.add(tf.matmul(layer_4,weights['h5']),biases['b5'])
    layer_5=tf.nn.relu(layer_5)
    
    
    out_layer=tf.add(tf.matmul(layer_5,weights['out']),biases['out'])
    return out_layer

weights={
       
    'h1':tf.Variable(tf.truncated_normal([42,n_hidden_1])),
    'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'h5':tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_5])),
    'out':tf.Variable(tf.truncated_normal([n_hidden_5,2]))
}

biases={
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'b5':tf.Variable(tf.truncated_normal([n_hidden_5])),
    'out':tf.Variable(tf.truncated_normal([2]))
}
    
y=multilayer_perceptron(x,weights,biases)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))  
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess.run(tf.global_variables_initializer())                     
                     
    
    

In [18]:
mse_history=[]
cost_history=[]
accuracy_history=[]

In [19]:
training_epochs=1000

In [20]:
for epoch in range(training_epochs):
    sess.run(train_step,feed_dict={x:x_train,y_:y_train})
    cost=sess.run(cross_entropy,feed_dict={x:x_train,y_:y_train})
    cost_history.append(np.append(cost_history,cost))
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    pred_y=sess.run(y,feed_dict={x:x_test})
    mse=tf.reduce_mean(tf.square(pred_y-y_test))
    mse_=sess.run(mse)
    mse_history.append(mse_)
    accuracy=sess.run(accuracy,feed_dict={x:x_train,y_:y_train})
    accuracy_history.append(accuracy)
    
    print('epoch:',epoch,'-','cost:',cost,'mse: ',mse_,'-','-train accuracy:',accuracy)

epoch: 0 - cost: 9.24004 mse:  136.942631748 - -train accuracy: 0.507852
epoch: 1 - cost: 17.0569 mse:  347.58140078 - -train accuracy: 0.492148
epoch: 2 - cost: 8.58024 mse:  127.062143022 - -train accuracy: 0.507852
epoch: 3 - cost: 14.5427 mse:  278.52839411 - -train accuracy: 0.492148
epoch: 4 - cost: 8.82081 mse:  131.060037545 - -train accuracy: 0.507852
epoch: 5 - cost: 12.6862 mse:  232.004399542 - -train accuracy: 0.492148
epoch: 6 - cost: 9.26931 mse:  138.657144932 - -train accuracy: 0.507852
epoch: 7 - cost: 11.1412 mse:  195.310900791 - -train accuracy: 0.492148
epoch: 8 - cost: 9.78716 mse:  147.933179215 - -train accuracy: 0.507852
epoch: 9 - cost: 9.75498 mse:  164.514362978 - -train accuracy: 0.492148
epoch: 10 - cost: 10.3139 mse:  158.001868568 - -train accuracy: 0.507852
epoch: 11 - cost: 8.50033 mse:  138.919446629 - -train accuracy: 0.492148
epoch: 12 - cost: 10.8092 mse:  168.540468024 - -train accuracy: 0.507852
epoch: 13 - cost: 7.41344 mse:  119.536690462 - -t

epoch: 112 - cost: 5.8407 mse:  120.896395714 - -train accuracy: 0.546713
epoch: 113 - cost: 3.79512 mse:  114.461293191 - -train accuracy: 0.52116
epoch: 114 - cost: 5.73659 mse:  119.54000648 - -train accuracy: 0.549108
epoch: 115 - cost: 3.73392 mse:  113.717422722 - -train accuracy: 0.524355
epoch: 116 - cost: 5.63398 mse:  118.207849354 - -train accuracy: 0.552569
epoch: 117 - cost: 3.67733 mse:  113.097783514 - -train accuracy: 0.527282
epoch: 118 - cost: 5.53691 mse:  116.971316209 - -train accuracy: 0.554964
epoch: 119 - cost: 3.62324 mse:  112.477349497 - -train accuracy: 0.53021
epoch: 120 - cost: 5.44551 mse:  115.822983913 - -train accuracy: 0.556561
epoch: 121 - cost: 3.56914 mse:  111.878677075 - -train accuracy: 0.531275
epoch: 122 - cost: 5.35534 mse:  114.665091546 - -train accuracy: 0.560021
epoch: 123 - cost: 3.5164 mse:  111.190286622 - -train accuracy: 0.532606
epoch: 124 - cost: 5.27119 mse:  113.552971891 - -train accuracy: 0.561884
epoch: 125 - cost: 3.46576 mse

epoch: 222 - cost: 3.06093 mse:  82.2360579073 - -train accuracy: 0.623902
epoch: 223 - cost: 2.08105 mse:  86.9668169501 - -train accuracy: 0.576258
epoch: 224 - cost: 3.03086 mse:  81.8172007175 - -train accuracy: 0.625233
epoch: 225 - cost: 2.06026 mse:  86.6490014086 - -train accuracy: 0.576524
epoch: 226 - cost: 2.99771 mse:  81.3873816022 - -train accuracy: 0.625765
epoch: 227 - cost: 2.04145 mse:  86.3803243668 - -train accuracy: 0.577588
epoch: 228 - cost: 2.96827 mse:  80.9896901854 - -train accuracy: 0.627096
epoch: 229 - cost: 2.02185 mse:  86.1110167143 - -train accuracy: 0.576524
epoch: 230 - cost: 2.93909 mse:  80.5932448209 - -train accuracy: 0.628959
epoch: 231 - cost: 2.00137 mse:  85.8430539107 - -train accuracy: 0.577056
epoch: 232 - cost: 2.90905 mse:  80.185223144 - -train accuracy: 0.629758
epoch: 233 - cost: 1.98097 mse:  85.5860875675 - -train accuracy: 0.578387
epoch: 234 - cost: 2.87952 mse:  79.7816013316 - -train accuracy: 0.630024
epoch: 235 - cost: 1.96136

epoch: 332 - cost: 2.05729 mse:  70.1043406065 - -train accuracy: 0.652382
epoch: 333 - cost: 1.44453 mse:  76.6705708486 - -train accuracy: 0.593825
epoch: 334 - cost: 2.04552 mse:  69.9302391202 - -train accuracy: 0.652382
epoch: 335 - cost: 1.43922 mse:  76.4959987307 - -train accuracy: 0.594091
epoch: 336 - cost: 2.03765 mse:  69.8067686438 - -train accuracy: 0.652382
epoch: 337 - cost: 1.43152 mse:  76.3220584984 - -train accuracy: 0.59489
epoch: 338 - cost: 2.02081 mse:  69.6217068469 - -train accuracy: 0.653181
epoch: 339 - cost: 1.42421 mse:  76.1460286193 - -train accuracy: 0.595156
epoch: 340 - cost: 2.00771 mse:  69.4815049744 - -train accuracy: 0.653447
epoch: 341 - cost: 1.41758 mse:  76.0065011495 - -train accuracy: 0.59489
epoch: 342 - cost: 1.99712 mse:  69.3655147196 - -train accuracy: 0.653713
epoch: 343 - cost: 1.41093 mse:  75.8665546522 - -train accuracy: 0.59489
epoch: 344 - cost: 1.98601 mse:  69.2545003066 - -train accuracy: 0.653979
epoch: 345 - cost: 1.4037 ms

epoch: 442 - cost: 1.55934 mse:  71.3812191396 - -train accuracy: 0.672345
epoch: 443 - cost: 1.14246 mse:  77.7508041681 - -train accuracy: 0.617514
epoch: 444 - cost: 1.5524 mse:  71.571778852 - -train accuracy: 0.672611
epoch: 445 - cost: 1.13864 mse:  77.9321418724 - -train accuracy: 0.61778
epoch: 446 - cost: 1.54616 mse:  71.7391254862 - -train accuracy: 0.672877
epoch: 447 - cost: 1.13446 mse:  78.0945285776 - -train accuracy: 0.61778
epoch: 448 - cost: 1.5404 mse:  71.8992699115 - -train accuracy: 0.673143
epoch: 449 - cost: 1.13116 mse:  78.2620298508 - -train accuracy: 0.617248
epoch: 450 - cost: 1.53518 mse:  72.0852983208 - -train accuracy: 0.67341
epoch: 451 - cost: 1.12651 mse:  78.4335827319 - -train accuracy: 0.617514
epoch: 452 - cost: 1.52673 mse:  72.2427048296 - -train accuracy: 0.67341
epoch: 453 - cost: 1.12234 mse:  78.6191530621 - -train accuracy: 0.617514
epoch: 454 - cost: 1.51991 mse:  72.429258063 - -train accuracy: 0.67341
epoch: 455 - cost: 1.11816 mse:  7

epoch: 552 - cost: 1.26575 mse:  79.3305642677 - -train accuracy: 0.686452
epoch: 553 - cost: 0.965839 mse:  85.6510655101 - -train accuracy: 0.626031
epoch: 554 - cost: 1.26203 mse:  79.4179724578 - -train accuracy: 0.686452
epoch: 555 - cost: 0.963512 mse:  85.7295840322 - -train accuracy: 0.626298
epoch: 556 - cost: 1.25764 mse:  79.4967486864 - -train accuracy: 0.686984
epoch: 557 - cost: 0.960846 mse:  85.7940319951 - -train accuracy: 0.626564
epoch: 558 - cost: 1.25274 mse:  79.5706629004 - -train accuracy: 0.687517
epoch: 559 - cost: 0.958282 mse:  85.8531816453 - -train accuracy: 0.626298
epoch: 560 - cost: 1.2492 mse:  79.6523759459 - -train accuracy: 0.68725
epoch: 561 - cost: 0.95674 mse:  85.9139004149 - -train accuracy: 0.626564
epoch: 562 - cost: 1.24731 mse:  79.7510986953 - -train accuracy: 0.68725
epoch: 563 - cost: 0.954567 mse:  86.0129126665 - -train accuracy: 0.627096
epoch: 564 - cost: 1.24252 mse:  79.8223411626 - -train accuracy: 0.68725
epoch: 565 - cost: 0.951

epoch: 662 - cost: 1.03563 mse:  84.5493681829 - -train accuracy: 0.701091
epoch: 663 - cost: 0.836733 mse:  90.3188583514 - -train accuracy: 0.647325
epoch: 664 - cost: 1.0314 mse:  84.66195877 - -train accuracy: 0.700825
epoch: 665 - cost: 0.834181 mse:  90.4103766555 - -train accuracy: 0.647591
epoch: 666 - cost: 1.02728 mse:  84.7637179401 - -train accuracy: 0.701357
epoch: 667 - cost: 0.832085 mse:  90.4870264874 - -train accuracy: 0.648124
epoch: 668 - cost: 1.02378 mse:  84.8555252004 - -train accuracy: 0.702156
epoch: 669 - cost: 0.829999 mse:  90.5592233308 - -train accuracy: 0.647591
epoch: 670 - cost: 1.02037 mse:  84.9374377194 - -train accuracy: 0.702688
epoch: 671 - cost: 0.828217 mse:  90.6261563011 - -train accuracy: 0.647857
epoch: 672 - cost: 1.01721 mse:  85.0177242567 - -train accuracy: 0.703221
epoch: 673 - cost: 0.826428 mse:  90.7177973836 - -train accuracy: 0.64839
epoch: 674 - cost: 1.01408 mse:  85.1138119785 - -train accuracy: 0.703487
epoch: 675 - cost: 0.82

epoch: 771 - cost: 0.738997 mse:  96.5372731834 - -train accuracy: 0.66223
epoch: 772 - cost: 0.864682 mse:  91.4603246676 - -train accuracy: 0.713601
epoch: 773 - cost: 0.737405 mse:  96.6348963831 - -train accuracy: 0.662497
epoch: 774 - cost: 0.862023 mse:  91.5551299292 - -train accuracy: 0.7144
epoch: 775 - cost: 0.735999 mse:  96.7069895643 - -train accuracy: 0.663029
epoch: 776 - cost: 0.859957 mse:  91.6502270558 - -train accuracy: 0.714932
epoch: 777 - cost: 0.734726 mse:  96.7882472222 - -train accuracy: 0.663561
epoch: 778 - cost: 0.857921 mse:  91.7381915651 - -train accuracy: 0.714932
epoch: 779 - cost: 0.733565 mse:  96.87260265 - -train accuracy: 0.664626
epoch: 780 - cost: 0.855821 mse:  91.8324527348 - -train accuracy: 0.714932
epoch: 781 - cost: 0.732157 mse:  96.9357942165 - -train accuracy: 0.664626
epoch: 782 - cost: 0.853466 mse:  91.9165699403 - -train accuracy: 0.714932
epoch: 783 - cost: 0.730886 mse:  97.015402692 - -train accuracy: 0.663828
epoch: 784 - cost:

epoch: 880 - cost: 0.760591 mse:  93.9032219113 - -train accuracy: 0.725047
epoch: 881 - cost: 0.671834 mse:  98.1006907843 - -train accuracy: 0.67128
epoch: 882 - cost: 0.758851 mse:  93.8777760268 - -train accuracy: 0.725313
epoch: 883 - cost: 0.670754 mse:  98.0553365009 - -train accuracy: 0.671813
epoch: 884 - cost: 0.757363 mse:  93.8444839196 - -train accuracy: 0.725047
epoch: 885 - cost: 0.669683 mse:  97.9765674141 - -train accuracy: 0.672877
epoch: 886 - cost: 0.755759 mse:  93.8080333809 - -train accuracy: 0.725313
epoch: 887 - cost: 0.668353 mse:  97.910283283 - -train accuracy: 0.673942
epoch: 888 - cost: 0.753644 mse:  93.7748065052 - -train accuracy: 0.725313
epoch: 889 - cost: 0.667255 mse:  97.8387763688 - -train accuracy: 0.674474
epoch: 890 - cost: 0.752195 mse:  93.7176134567 - -train accuracy: 0.725047
epoch: 891 - cost: 0.666321 mse:  97.7557233908 - -train accuracy: 0.674208
epoch: 892 - cost: 0.750996 mse:  93.6580464096 - -train accuracy: 0.72478
epoch: 893 - co

epoch: 989 - cost: 0.618234 mse:  98.294133046 - -train accuracy: 0.688581
epoch: 990 - cost: 0.678865 mse:  94.9385162386 - -train accuracy: 0.736226
epoch: 991 - cost: 0.617311 mse:  98.3157544048 - -train accuracy: 0.68938
epoch: 992 - cost: 0.677455 mse:  94.960168238 - -train accuracy: 0.736758
epoch: 993 - cost: 0.616359 mse:  98.3310710166 - -train accuracy: 0.690445
epoch: 994 - cost: 0.676074 mse:  95.0025157792 - -train accuracy: 0.736758
epoch: 995 - cost: 0.615464 mse:  98.3508679893 - -train accuracy: 0.690977
epoch: 996 - cost: 0.674762 mse:  95.0278036038 - -train accuracy: 0.737024
epoch: 997 - cost: 0.614544 mse:  98.3618827825 - -train accuracy: 0.691509
epoch: 998 - cost: 0.673484 mse:  95.0697573212 - -train accuracy: 0.73729
epoch: 999 - cost: 0.613703 mse:  98.4101008863 - -train accuracy: 0.693639


In [21]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('test accuracy:',sess.run(accuracy,feed_dict={x:x_test,y_:y_test}))

test accuracy: 0.683955


In [22]:
pred_y=sess.run(y,feed_dict={x:x_test})
mse=tf.reduce_mean(tf.square(pred_y-y_test))
print('mse:%.4f'%sess.run(mse))

mse:98.4101
